# 1.3.1 Разбиения и Валидация (Splitting and Validation)

Правильное разбиение данных гарантирует, что оценка качества модели (метрики) будет **непредвзятой** и отразит способность модели к **обобщению** на новых, невидимых данных.

---

### Основной Принцип Разбиения: Hold-out

| Набор | Назначение | Доля (Пример) | Ключевой принцип |
| :--- | :--- | :--- | :--- |
| **Train (Обучающий)** | Используется для **обучения** модели (настройки весов). | 60–80% | **Самый большой** набор. |
| **Validation (Валидационный)** | Используется для **настройки гиперпараметров** и **ранней остановки** (early stopping). | 10–20% | Модель его **не видит** в процессе обучения, но вы его используете для **выбора** лучшей модели. |
| **Test (Тестовый)** | Используется **только один раз** в конце, для получения **финальной, непредвзятой** оценки качества. | 10–20% | **Нельзя использовать** для настройки модели. |

---

### Методы Кросс-Валидации (Cross-Validation)

Кросс-валидация используется для получения **более стабильной** оценки качества модели на этапе настройки гиперпараметров (заменяет одиночный Validation set).

| Метод | Назначение | Свойства | Когда использовать |
| :--- | :--- | :--- | :--- |
| **K-Fold** | Базовый метод. Данные делятся на **K** равных блоков (фолдов). Модель обучается K раз, каждый раз используя новый фолд как валидационный. | Уменьшает смещение оценки и обеспечивает высокую надежность. | **Общие задачи** (где нет сильной временной или групповой структуры). |
| **Stratified K-Fold** | Расширение K-Fold. Делит данные так, чтобы **пропорция классов** (0 и 1) в каждом фолде была такой же, как и во всем датасете. | **Обязателен** для задач **Классификации**, особенно при **несбалансированных классах**. | **Классификация** (бинарная/мультиклассовая). |
| **Group K-Fold** | Гарантирует, что все наблюдения, принадлежащие одной **группе** (`CustomerID`, `SessionID`), остаются в **одном фолде** (тренировочном или валидационном). | **Критически важен** для предотвращения **Data Leakage** (например, нельзя тестировать модель на том же клиенте, которого она видела в обучении). | Данные с естественными группами. |
| **Time-Series Split** | **Walk-Forward Validation**. Разбиение **по времени**. Фолд $i$ используется для обучения, а фолд $i+1$ — для валидации. | Предотвращает использование **будущих** данных для предсказания **прошлого** (защита от Temporal Leakage). | **Временные ряды**, финансовые или последовательные данные. |


# 1.3.2 Метрики Регрессии

Метрики регрессии оценивают, насколько близко предсказанное значение ($\hat{y}$) находится к истинному значению ($y$).

---

### Основные Метрики Ошибки

| Метрика | Формула | Свойства (Что минимизирует) | Когда использовать |
| :--- | :--- | :--- | :--- |
| **MAE (Mean Absolute Error)** | $\frac{1}{N}\sum_{i=1}^{N} \|y_i - \hat{y}_i\|$ | Сумма **абсолютных** ошибок. Ошибка растет **линейно**. **Устойчива к выбросам**. | Когда выбросы не являются критическими, и вы хотите, чтобы все ошибки имели равный вес. |
| **RMSE (Root Mean Squared Error)** | $\sqrt{\frac{1}{N}\sum_{i=1}^{N} (y_i - \hat{y}_i)^2}$ | Сумма **квадратов** ошибок. Ошибка растет **квадратично**. **Чувствительна к выбросам** (наказывает большие ошибки сильнее). | Когда большие ошибки недопустимы (например, в финансах, инженерии) и должны быть строго минимизированы. |
| **$R^2$ (Коэффициент детерминации)** | $1 - \frac{\sum_{i=1}^{N} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{N} (y_i - \bar{y})^2}$ | Доля дисперсии целевой переменной, объясненная моделью, относительно базовой модели (простого среднего $\bar{y}$). | **Общая оценка качества**. Диапазон $[-\infty; 1]$. Значение 1 — идеальная модель. Значение < 0 означает, что модель хуже, чем простое предсказание среднего. |

### Относительные Метрики

| Метрика | Формула | Свойства | Когда использовать |
| :--- | :--- | :--- | :--- |
| **MAPE (Mean Absolute Percentage Error)** | $\frac{100\%}{N}\sum_{i=1}^{N} \left\| \frac{y_i - \hat{y}_i}{y_i} \right\| $ | Ошибка выражена в **процентах**. Удобна для сравнения прогнозов с разными масштабами. | **Прогнозирование спроса, продаж.** Модель сильно штрафуется, если истинное значение ($y_i$) близко к нулю. |
| **sMAPE (Symmetric MAPE)** |$ \frac{100\%}{N}\sum_{i=1}^{N} \frac{\|y_i - \hat{y}_i\|}{(\|y_i\| + \|\hat{y}_i\|)/2} $| Более **симметричный** вариант MAPE. Делит ошибку на среднее значение истинного и предсказанного. | Предпочтительна перед MAPE, так как штрафует ошибки более равномерно и не имеет проблемы деления на ноль. |

### Метрика Квантильной Регрессии

| Метрика | Формула | Свойства | Когда использовать |
| :--- | :--- | :--- | :--- |
| **Pinball Loss (Квантильная)** | $(y - \hat{y}) \cdot \tau$ при $y > \hat{y}$<br>$(\hat{y} - y) \cdot (1 - \tau)$ при $y \le \hat{y}$ | **Асимметричная** функция потерь. Наказывает промах в одном направлении сильнее, чем в другом, в зависимости от выбранного **квантиля $\tau$**. | **Прогнозирование рисков/диапазонов**. Например, для $\tau=0.95$ модель будет минимизировать случаи, когда истинное значение **превышает** 95-й квантиль предсказания. |
